In [ ]:
# Run the code without the pump, and calculate the steady-state induced by the probe.

# The scattering rate is given by a sum of a term proportional to the Fermi energy
# and a term proportional to the inverse of the Fermi energy.
# Verify the dependence of the THG on the two proportionality coefficients.

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import json
from datetime import datetime
import multiprocessing as mp
import glob

In [ ]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

In [ ]:
import models.graphene_thermodynamics_v2 as gt
from models.cooling_phonons_v3 import CoolingPhonons
from models.graphene_optics import GrapheneTHG

In [ ]:
# Load interpolation data for the calculation of chemical potentials.
mm = gt.mu_func_use_2d_interpolation(load_from="mu_table.pkl")

In [ ]:
# Return mantissa and exponent base 10.
def frexp10(x):
    exp = int(math.log10(x))
    return (x / 10**exp, exp)

# Coefficients: constant

In [ ]:
# Run the evolution for several Fermi energies and save the values of the THG.

def oneRun(x):
    thg_list = []
    for ieF,eF in enumerate(x["eF_list"]):
        sysparams = {**x["sysparams"], "eF": eF}
        dyn = CoolingPhonons(**sysparams)
        dyn.run(**x["runparams"])
        thg_list.append(dyn.p["etaTHG_avg"])
        
        # Save all time-evolutions.  Takes up much space.
        if False:
            runlabel="%02d-%02d-%02d" % (x["i1"], x["i2"], ieF)
            with open("%s/params-%s.json" % (x["saveDir"], runlabel), "w") as f:
                json.dump({"sysparams": x["sysparams"], "runparams": runparams}, f, indent=4)
            np.savetxt("%s/dynamics-%s.csv" % (x["saveDir"], runlabel), dyn.dynamics_m, delimiter=",")
            np.savetxt("%s/thg-%s.csv" % (x["saveDir"], runlabel), np.c_[dyn.p["pr_tt"],dyn.p["etaTHG"]], delimiter=",")
    thg_list = np.array(thg_list)

    r = (x["i1"], thg_list)
    return r

In [ ]:
# Dir to save results.
run_label = "coeff_const_30K"
time_stamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
saveDir = "./data/06/%s_%s" % (time_stamp, run_label)
os.mkdir(saveDir)


# System's parameters common to all runs.
sysparams = {
    "tempK_eq": 30.0,
    "tau_ph": 1200.0,
    "pu_fluenceSI": 0.0,
    "pu_dt": 50.0,
    "pu_eph": 1.200,
    "pr_delay": 50.0,
    "pr_fluenceSI": 130.0,
    "pr_dt": 150.0,
    "pr_eph": 0.320,
    "nsub": 1.4,  # hBN encapsulated with SiO2 around
    "ntop": 1.4,
    "gammaProp": 0.0,
    "gammaInv": 0.0,
    "dtdb": 200.0,
    "ph_en_frac": 1.0,
    "dtdb_cutoff_dens": None
}

runparams = {
    "tmax": 250.0,  # integrate up to a bit later than the end of the probe pulse.
    "tnum": 51,  # need a finer time mesh, possibly aligned with the pulse
    "dt_approx": 5.0,
    "print_time": False    
}

# List of Fermi energies [eV].
eF_list = [0.050, 0.100, 0.150, 0.200, 0.250]
# List of the constant coefficient in the expression of Gamma.
gammaConstList = np.logspace(-6.0, -2.0, 41)

# Create a list of parameters, one for each run.
params_list = []
for i1,gammaConst in enumerate(gammaConstList):
    params_list.append({"i1": i1, "saveDir": saveDir, "eF_list": eF_list,
                        "sysparams": {**sysparams, "gammaConst": gammaConst},
                        "runparams": runparams})

# Temperature as string.
tempK = "%.0f" % sysparams["tempK_eq"]


In [ ]:
# Run the calculation.
with mp.Pool(24) as p:
    res_list = p.map(oneRun, params_list)

In [ ]:
# Collect the results.
thg_list = np.zeros((len(gammaConstList),len(eF_list)))
for res in res_list:
    thg_list[res[0],:] = res[1]

In [ ]:
# Save the results.
with open("%s/params.json" % saveDir, "w") as f:
    json.dump({"sysparams": sysparams, "runparams": runparams}, f)
    
np.savetxt("%s/eF_list.csv" % saveDir, eF_list, delimiter=", ")
np.savetxt("%s/gammaConst_list.csv" % saveDir, gammaConstList, delimiter=", ")
np.savetxt("%s/thg_list.csv" % saveDir, thg_list, delimiter=", ")

In [ ]:
# Reload results if needed.
if False:
    saveDir = "./data/06/2023-08-14-15-09-04_coeff_const_300K"
    #saveDir = "./data/06/2023-08-13-13-10-45_coeff_const_30K"
    eF_list = np.loadtxt("%s/eF_list.csv" % saveDir, delimiter=", ")
    gammaConstList = np.loadtxt("%s/gammaConst_list.csv" % saveDir, delimiter=", ")
    thg_list = np.loadtxt("%s/thg_list.csv" % saveDir, delimiter=", ")
    tempK = "300"

In [ ]:
# Plot the THG as a function of the Fermi energy for selected values of
# the constant coefficient in Gamma.
ii = [10, 20, 30, 40]
mpl.style.use(["classic", "latex"])
cmap = mpl.cm.get_cmap('gnuplot_r')
plt.figure(figsize=(3.5,3.0), frameon=True, dpi=150)
plt.axes([0.2, 0.2, 0.45, 0.7])
for i,i1 in enumerate(ii):
    plt.plot(eF_list, np.log10(thg_list[i1,:]), "-o", ms=5.0,
             color=cmap((i+1)/len(ii)),
             label=r"$%.6f$" % gammaConstList[i1])
plt.xlim([0.050, 0.250])
plt.ylim([-10.0, -7.0])
plt.xlabel(r"$\varepsilon_{\rm F}~[{\rm eV}]$")
plt.ylabel(r"$\log_{10}({\rm THG})$")
plt.title(r"\scriptsize $T_{\rm L} = %s~{\rm K}$" % tempK)
plt.legend(frameon=False, loc="upper left", bbox_to_anchor=[1.0, 0.2, 0.3, 0.5], fontsize=9, numpoints=1, title=r"$\Gamma_{0}(T_{\rm L})$")
plt.savefig("%s/thg_vs_ef.png" % saveDir, dpi=300)

# Coefficients: proportional and inversely proportional

In [ ]:
# Run the evolution for several Fermi energies and save the values of the THG.

def oneRun(x):
    thg_list = []
    for ieF,eF in enumerate(x["eF_list"]):
        sysparams = {**x["sysparams"], "eF": eF}
        dyn = CoolingPhonons(**sysparams)
        dyn.run(**x["runparams"])
        thg_list.append(dyn.p["etaTHG_avg"])
        
        # Save all time-evolutions.  Takes up much space.
        if False:
            runlabel="%02d-%02d-%02d" % (x["i1"], x["i2"], ieF)
            with open("%s/params-%s.json" % (x["saveDir"], runlabel), "w") as f:
                json.dump({"sysparams": x["sysparams"], "runparams": runparams}, f, indent=4)
            np.savetxt("%s/dynamics-%s.csv" % (x["saveDir"], runlabel), dyn.dynamics_m, delimiter=",")
            np.savetxt("%s/thg-%s.csv" % (x["saveDir"], runlabel), np.c_[dyn.p["pr_tt"],dyn.p["etaTHG"]], delimiter=",")
    thg_list = np.array(thg_list)

    r = (x["i1"], x["i2"], thg_list)
    return r

In [ ]:
# System's parameters common to all runs.
sysparams = {
    "tempK_eq": 30.0,
    "tau_ph": 1200.0,
    "pu_fluenceSI": 0.0,
    "pu_dt": 50.0,
    "pu_eph": 1.200,
    "pr_delay": 50.0,
    "pr_fluenceSI": 130.0,
    "pr_dt": 150.0,
    "pr_eph": 0.320,
    "nsub": 1.4,  # hBN encapsulated with SiO2 around
    "ntop": 1.4,
    "gammaConst": 0.0,
    "dtdb": 200.0,
    "ph_en_frac": 1.0,
    "dtdb_cutoff_dens": None
}

runparams = {
    "tmax": 250.0,  # integrate up to a bit later than the end of the probe pulse.
    "tnum": 51,  # need a finer time mesh, possibly aligned with the pulse
    "dt_approx": 5.0,
    "print_time": False    
}

# List of Fermi energies [eV].
eF_list = [0.050, 0.100, 0.150, 0.200, 0.250]
# List of the two coefficients in the expression of Gamma.
gammaPropList = np.logspace(-2.0, 0.0, 21)
gammaInvList = np.logspace(-2.0, -1.0, 31)

# Create a list of parameters, one for each run.
params_list = []
for i1,gammaProp in enumerate(gammaPropList):
    for i2,gammaInv in enumerate(gammaInvList):
        params_list.append({"i1": i1, "i2": i2, "saveDir": saveDir, "eF_list": eF_list,
                            "sysparams": {**sysparams, "gammaProp": gammaProp, "gammaInv": gammaInv},
                            "runparams": runparams})

In [ ]:
# Run the calculation.
with mp.Pool(24) as p:
    res_list = p.map(oneRun, params_list)

In [ ]:
# Collect the results.
thg_3d = np.zeros((len(gammaPropList),len(gammaInvList),len(eF_list)))
for res in res_list:
    thg_3d[res[0],res[1],:] = res[2]
# Flatten the results into a dataframe.
# Each row contains the scan over the Fermi energy
# and refers to the values of gammaProp and gammaInv
# unrolled with gammaInv increasing faster.
thg_list = []
for i1 in range(len(gammaPropList)):
    for i2 in range(len(gammaInvList)):
        thg_list.append(thg_3d[i1,i2,:])
thg_list = np.array(thg_list)

In [ ]:
# Save the results.
np.savetxt("%s/eF_list.csv" % saveDir, eF_list, delimiter=", ")
np.savetxt("%s/gammaProp_list.csv" % saveDir, gammaPropList, delimiter=", ")
np.savetxt("%s/gammaInv_list.csv" % saveDir, gammaInvList, delimiter=", ")
np.savetxt("%s/thg_list.csv" % saveDir, thg_list, delimiter=", ")


In [ ]:
# Load previous results.

In [ ]:
# Plot the THG as a function of the Fermi energy for selected values
# of the other two parameters.
i1 = 20
i2 = 10
# ii = [(i1, 0), (i1, 9), (i1, 12), (i1, 20)]
ii = [(0, i2), (10, i2), (15, i2), (20, i2)]
mpl.style.use(["classic", "latex"])
cmap = mpl.cm.get_cmap('gnuplot_r')
plt.figure(figsize=(3.5,3.0), frameon=True, dpi=150)
plt.axes([0.2, 0.2, 0.7, 0.7])
for i,(i1,i2) in enumerate(ii):
    plt.plot(eF_list, np.log10(thg_3d[i1,i2,:]), "-ok", ms=5.0,
             color=cmap((i+1)/len(ii)),
             label=r"%6.2e %6.2e" % (gammaPropList[i1], gammaInvList[i2]))
plt.xlim([0.050, 0.250])
plt.ylim([-10.0, -7.0])
plt.xlabel(r"$\varepsilon_{\rm F}~[{\rm eV}]$")
plt.ylabel(r"$\log_{10}({\rm THG})$")
plt.legend(frameon=False,loc="upper left", fontsize=9)
plt.savefig("%s/thg_vs_ef_04.png" % saveDir, dpi=300)